In [5]:
from lyra import Client
from lyra.parser import parse_form
from lyra.transformer.transformer import transform_form

from fundar import json
from glob import glob
from tqdm.auto import tqdm

forms = glob('../data/*-form.json')

form = forms[0]

ast = parse_form(json.load(form))
transform_form(ast)

['variable: str',
 'spatial_aggregation: str',
 'energy_product_type: str',
 'temporal_aggregation: str',
 'experiment: str',
 'rcm: str',
 'gcm: str',
 'ensemble_member: str']